Predicting Estimated Salary(continous/Numerical Feature)

In [14]:
import numpy as np 
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle 

In [15]:
data = pd.read_csv("Churn_Modelling.csv")

In [16]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [18]:
#preprocessing the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [19]:
#label encoder for gender
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [20]:
#OHE geography col
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [21]:
#concatenate geo with df
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [22]:
#split data into independent and dependent features(target)
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [23]:
#split data into train test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [24]:
#standard scaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [25]:
#save the pckl files(label,ohe,scaled)
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

Train ANN with Regression Problem Statement

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [29]:
#Build the model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1
    Dense(32,activation='relu'), #HL2
    Dense(1) #op layer default activation is linear(specific to regression)
])

#Compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#set up tensorboard callback
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [35]:
#set up early stopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [36]:
#Train the model
history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 100367.9609 - mae: 100367.9609 - val_loss: 98483.7812 - val_mae: 98483.7812
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99524.8672 - mae: 99524.8672 - val_loss: 96801.4531 - val_mae: 96801.4531
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96629.5625 - mae: 96629.5625 - val_loss: 92586.6797 - val_mae: 92586.6797
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91010.1562 - mae: 91010.1562 - val_loss: 85645.0938 - val_mae: 85645.0938
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 82941.2422 - mae: 82941.2422 - val_loss: 76799.3359 - val_mae: 76799.3359
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 73620.4766 - mae: 73620.4766 - val_loss: 67816.4609 - val_mae: 67816.4609
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 64824.3359 - mae: 64824.335

In [37]:
%load_ext tensorboard

In [40]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 15696), started 0:00:13 ago. (Use '!kill 15696' to kill it.)

In [41]:
#Evaluate the model for test data 
test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

63/63 [==============================] - 0s 1ms/step - loss: 50261.1445 - mae: 50261.1445
Test Loss: 50261.14453125
Test MAE: 50261.14453125


In [42]:
model.save('regression_model.h5')

c:\Users\hp\Desktop\DL using ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
